In [10]:
work_path = r'../../../'
quetzal_path = work_path + r'quetzal_santo_domingo/'

In [11]:
from tqdm import tqdm

In [12]:
import pandas as pd
    
parameter_frame = pd.read_excel(quetzal_path + r'inputs/parametros.xlsx').set_index(['category','parameter'])
for c in parameter_frame.columns:
    parent = parameter_frame[c][('general', 'parent')]
    try:
        parent = parent.iloc[0]
    except AttributeError:# 'str' object has no attribute 'iloc'
        pass
    parameter_frame[c] = parameter_frame[c].fillna(parameter_frame[parent])

scenario_list = list(parameter_frame.columns)

In [13]:
pool = {}

In [14]:
df_dict = pd.read_excel(quetzal_path + 'model/ref_18' + '/stacks.xlsx', sheet_name=None)
for key in df_dict.keys():
    pool[key] = []
for scenario in tqdm(scenario_list):
    df_dict = pd.read_excel(quetzal_path + r'model/' + scenario + '/stacks.xlsx', sheet_name=None)
    for key, value in df_dict.items():

        value['scenario'] = scenario
        col = [c for c in value.columns if 'scenario' not in c]
        col.insert(-1, 'scenario')
        value = value[col]
        pool[key].append(value)


100%|██████████| 5/5 [00:00<00:00,  5.94it/s]


In [15]:
stacks = {k: pd.concat(v) for k, v in pool.items()}

In [27]:
def densify(stack):
    stack = stacks['summary']
    c = stack.columns
    return stack.set_index(list(c[:-1]))[c[-1]].unstack()

In [28]:
dense_stacks = {k: densify(v) for k, v in stacks.items()}

In [ ]:
with pd.ExcelWriter(quetzal_path + r'outputs/'+ 'stacks.xlsx') as writer:  # doctest: +SKIP
    for name, stack in stacks.items():
        stack.to_excel(writer, sheet_name=name, index=False)

In [46]:
with pd.ExcelWriter(quetzal_path + r'outputs/'+ 'dense_stacks.xlsx') as writer:  # doctest: +SKIP
    for name, stack in dense_stacks.items():
        stack.reset_index().to_excel(writer, sheet_name=name, index=False)